In [5]:
from langchain.chat_models import ChatOpenAI
from langchain.prompts import ChatPromptTemplate
from langchain.schema import BaseOutputParser

chat = ChatOpenAI(model="gpt-3.5-turbo", temperature=0.5)

poem_template = ChatPromptTemplate.from_messages([
    ("system", "You are a poet who specializes in writing poems about programming languages. Write a poem that captures the essence and beauty of the language."),
    ("human", "Write a poem about {language_name}.")
])

explanation_template = ChatPromptTemplate.from_messages([
    ("system", "You are a literary critic specializing in the analysis of poetry. Explain the themes and emotions conveyed in the following poem about a programming language."),
    ("human", "{poem}")
])

class TextOutputParser(BaseOutputParser):
    def parse(self, text):
        return text.strip()

poem_chain = poem_template | chat | TextOutputParser()
explanation_chain = explanation_template | chat | TextOutputParser()

final_chain = poem_chain | explanation_chain
poem_chain.invoke({"language_name": "Python"})
explanation_chain.invoke({"poem": "Python is a programming language that is known for its simplicity and readability."})



"The poem about Python as a programming language conveys themes of simplicity, clarity, and efficiency. The choice of Python as the subject of the poem suggests a focus on the elegance and straightforwardness of the language. The poet likely appreciates Python for its clean syntax and ease of use, as well as its ability to express complex ideas in a concise manner.\n\nThe emotions conveyed in the poem are likely ones of admiration and respect for Python as a tool for creativity and problem-solving. The poet may feel a sense of joy and satisfaction in using Python to bring their ideas to life, and appreciate the way the language allows them to express themselves in a clear and efficient manner.\n\nOverall, the poem about Python as a programming language celebrates the beauty of simplicity and the power of clear communication, reflecting the poet's appreciation for a language that enables them to create and innovate with ease."